In [13]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import sys
import dateutil.parser

from __future__ import print_function

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
sys.path.insert(0, os.path.abspath('..'))

from UKMovementSensing import dataprep

## Filepaths

In [37]:
output_path = '/media/sf_VBox_Shared/London/raw/next161/'
annotations_path = output_path + 'tud_next161_deb.csv'
wearcodes_path = output_path + 'wearcodes.csv'

In [38]:
file_path = output_path + 'accelerometer_5second/'

## Run the preprocessing script

In [39]:
annotations = dataprep.process_annotations(annotations_path)
annotations_codes = dataprep.join_wearcodes(wearcodes_path, annotations)
dfs = dataprep.process_data(annotations_codes, file_path)
dataprep.save_merged(dfs, os.path.join(output_path, 'merged/'))
subsets = dataprep.take_subsequences(dfs)
dataprep.save_subsequences(subsets, os.path.join(output_path, 'subsets/'))

Number of missing start and end times: 0 0
Multiple end times per slot. first 10:
Slot
1     [04:10:00]
2     [04:20:00]
3     [04:30:00]
4     [04:40:00]
5     [04:50:00]
6     [05:00:00]
7     [05:10:00]
8     [05:20:00]
9     [05:30:00]
10    [05:40:00]
Name: end_time_time, dtype: object


TypeError: can't compare offset-naive and offset-aware datetimes

## Investigate annotations

In [ ]:
print(annotations.shape)
annotations.head()

In [ ]:
# To check: Do we have any gaps?
for i in range(1, annotations.shape[0]):
    if annotations['serflag'][i]==annotations['serflag'][i-1] and annotations['tud_day'][i]==annotations['tud_day'][i-1]:
        if (annotations['end_time'][i-1] != annotations['start_time'][i]):
            print(annotations.loc[[i-1, i],['start_time', 'end_time']])
            print(annotations['end_time'][i-1] - annotations['start_time'][i])

## Investigate join with wearcodes

In [ ]:
print(annotations_codes.shape)
annotations_codes.head()

## Investigate accelerometer data

In [ ]:
example_key = dfs.keys()[0]
binfile, day = example_key
df = dfs[example_key]
df.head()

In [ ]:
t = df.index[0]
print(t.tz)

## Investigate subsequences

In [ ]:
#Create boxplots for each sequence for the angles
#We expect x (and thus anglex) to be distributed either mostly on the negative or mostly on the positive half
fig, axes = plt.subplots(len(subsets.values()), figsize=(10, 50))
for i, dataset in enumerate(subsets.values()):
    non_sleeping_indices = dataset['act'] != 1.0
    non_sleeping = dataset[non_sleeping_indices]
    print(np.median(non_sleeping['anglex']), np.median(non_sleeping['angley']))
    axes[i].boxplot([non_sleeping['anglex'], non_sleeping['angley'], non_sleeping['anglez']], labels=['x', 'y', 'z']);

In [ ]:
subsets = dataprep.switch_positions(subsets)

In [ ]:
#Create boxplots for each sequence for the angles
#We expect x (and thus anglex) to be distributed either mostly on the negative or mostly on the positive half
fig, axes = plt.subplots(len(subsets.values()), figsize=(10, 50))
for i, dataset in enumerate(subsets.values()):
    non_sleeping_indices = dataset['act'] != 1.0
    non_sleeping = dataset[non_sleeping_indices]
    print(np.median(non_sleeping['anglex']), np.median(non_sleeping['angley']))
    axes[i].boxplot([non_sleeping['anglex'], non_sleeping['angley'], non_sleeping['anglez']], labels=['x', 'y', 'z']);